In [12]:
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')

In [13]:
data = pd.read_csv('recs_synthetic.csv')
data['rnk'] = data.groupby('user_id').cumcount()
data.head()

,user_id,score,target,total_actions,rnk
0,0,0.993668,1,17,0
1,0,0.896899,0,17,1
2,0,0.875116,1,17,2
3,0,0.844549,0,17,3
4,0,0.837955,0,17,4


In [17]:
def precision_at_k(df, k):
    df = df[df['rnk'] < k]
    df['pred'] = [1]*df.shape[0]
    df['tp'] = df['target'] & df['pred']
    df['fp'] = df[(df['target']!=df['pred'])&(df['pred']==1)]
    return df['tp'].sum()/(df['tp'].sum()+df['fp'].sum())

def recall_at_k(df, k):
    df['pred'] = df['rnk'].apply(lambda x: 1 if x < k else 0)
    df['tp'] = df['target'] & df['pred']
    agg_df = df[['user_id', 'tp', 'total_actions']].groupby('user_id')\
                .agg({'tp': 'sum', 'total_actions': 'max'}) # тут можно сразу tp поделить на total_actions, чтобы не делать 2 агрегации
    agg_df['recall'] = agg_df['tp']/agg_df['total_actions']
    return agg_df['recall'].mean()

In [4]:
x

In [5]:
def ndcg_at_k(df, k):
    df = df[df['rnk'] < k]
    df['dcg_i'] = 2**df['target']-1/df['rnk'].apply(lambda x: math.log(x + 2, 2))
    df = df.sort_values(['user_id', 'target'], ascending=[True, False])
    df['rnk_new'] = df.groupby('user_id').cumcount()
    df['idcg_i'] = 2**df['target']-1/df['rnk_new'].apply(lambda x: math.log(x + 2, 2))
    dcg_df = df[['user_id', 'dcg_i', 'idcg_i']].groupby('user_id').sum()
    dcg_df['ndcg'] = dcg_df['dcg_i']/dcg_df['idcg_i']
    return dcg_df['ndcg'].mean()

In [6]:
def mrr(df):
    df = df[df['target']==1]
    agg_df = df[['user_id', 'rnk']].groupby('user_id').min()
    agg_df['tmp'] = 1/(agg_df['rnk']+1)
    return agg_df.tmp.sum()/agg_df.shape[0]

In [18]:
print(f'precision@8: {precision_at_k(df=data.copy(), k=8)}')
print(f'precision@30: {precision_at_k(df=data.copy(), k=30)}')

ValueError: Wrong number of items passed 7, placement implies 1

In [8]:
print(f'recall@8: {recall_at_k(df=data.copy(), k=8)}')
print(f'recall@30: {recall_at_k(df=data.copy(), k=30)}')

recall@8: 0.1521432894570625
recall@30: 0.5879062358080964


In [9]:
print(f'MAP@8: {map_at_k(df=data.copy(), k=8)}')
print(f'MAP@30: {map_at_k(df=data.copy(), k=30)}')

MAP@8: 0.49745983037797686
MAP@30: 0.39767171242806565


In [10]:
print(f'MRR: {mrr(df=data.copy())}')

MRR: 0.5359987465750623


In [11]:
ndcg_at_k(df=data.copy(), k=8), ndcg_at_k(df=data.copy(), k=30)

(0.6563167070312262, 0.662153051871711)